In [9]:
!pip install spacy
!pip install transformers
!pip install nltk
!pip install PyPDF2
!pip install python-docx --upgrade

In [10]:
import PyPDF2
import docx
import spacy
from transformers import pipeline
import os
nlp = spacy.load("en_core_web_sm")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
def read_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text.strip()
def read_docx(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text.strip()
def preprocess_text(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    return sentences
def extract_entities(text):
    doc = nlp(text)
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    return entities
def summarize_text(text):
    if len(text) > 1024:
        text = text[:1024]
    summary = summarizer(text, max_length=150, min_length=40, do_sample=False)
    return summary[0]['summary_text']
def process_legal_document(text):
    sentences = preprocess_text(text)
    entities = extract_entities(text)
    summary = summarize_text(text)
    return {
        "Sentences": sentences,
        "Entities": entities,
        "Summary": summary
    }
def main():
    file_path = input("Enter the path of legal file: ")
    if not os.path.isfile(file_path):
        print("Cannot Find!")
        return
    _, file_extension = os.path.splitext(file_path)
    if file_extension.lower() == '.pdf':
        document_text = read_pdf(file_path)
    elif file_extension.lower() == '.docx':
        document_text = read_docx(file_path)
    else:
        print("Only PDF or MS doc/docx files supported!!!")
        return
    result = process_legal_document(document_text)
    print("Extracted legal sentences:", result["Sentences"])
    print("Legal named entities:", result["Entities"])
    print("Final Summary:", result["Summary"])
if __name__ == "__main__":
    main()

Enter the path of legal file: /content/legal.docx
Extracted legal sentences: ['Free legal information disclaimer: cover\n1.\t', 'This template legal document was produced and published by Docular Limited.\n', '2.\t', 'We control the copyright in this template, and you may only use this template in accordance with the licensing provisions in our terms and conditions.', 'Those licensing provisions include an obligation to retain the attribution / credit incorporated into the template.\n', '3.\tYou will need to edit this template before use.', 'Guidance notes to help you do so are set out at the end of the template.', 'During the editing process, you should delete those guidance notes and this cover sheet.', 'Square brackets in the body of the document indicate areas that require editorial attention.', '"ORs" in the body of the document indicate alternative provisions.', 'By the end of the editing process, there should be no square brackets left in the body of the document, and only one a